In [2]:
import pandas as pd
from zipfile import ZipFile
import os
import json
import shutil
from jiwer import wer, cer

import glob
import subprocess
import tarfile
import wget
import copy
from omegaconf import OmegaConf, open_dict

import wandb
from tqdm.auto import tqdm

from collections import defaultdict

import re
import unicodedata

import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager
from nemo.collections.nlp.models import PunctuationCapitalizationModel

import torch
import pytorch_lightning as ptl

[NeMo W 2022-12-16 18:16:01 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-12-16 18:16:02 nemo_logging:349] /home/a.lokis/miniconda3/envs/asr_env/lib/python3.9/site-packages/torch/jit/annotations.py:299: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
      warnings.warn("TorchScript will treat type annotations of Tensor "
    
[NeMo W 2022-12-16 18:16:03 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-12-16 18:16:04 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully su

# LM model KenLM

# Punctuation and Capitalization Model

### Build config

In [3]:
def build_trainer_and_model(restore_model_path=None):
    
    config_path = 'CapPunkt_configs/punctuation_capitalization_lexical_audio_config.yaml'
    config = OmegaConf.load(config_path) 

    accelerator = 'gpu' 
    config.trainer.devices = [2]
    config.trainer.accelerator = accelerator

    config.trainer.max_epochs = 100

    config.trainer.strategy = None
    config.exp_dir = f'experiments/'
    config.name = f"LexicalCapPunkt-LexCapPunkt"
    config.checkpoint_callback_params=exp_manager.CallbackParams(
                                               monitor="val_loss",
                                               mode="min",
                                               always_save_nemo=True,
                                               save_best_model=True),
    config.exp_manager.create_wandb_logger = True
    config.exp_manager.wandb_logger_kwargs = {'name': 'test_LCP',
                                              'project': 'asr_experiments', 
                                              'log_model': 'all'}
    config.exp_manager.explicit_log_dir='Punctuation_And_Capitalization_Lexical_Audio'

    trainer = ptl.Trainer(**config.trainer)
    
    PRETRAINED_BERT_MODEL = "bert-base-uncased"
    PRETRAINED_ASR_MODEL = 'stt_ru_quartznet15x5'

    config.model.train_ds.ds_item = 'data_for_CapPunkt'
    config.model.validation_ds.ds_item = 'data_for_CapPunkt'
    config.model.test_ds.ds_item = 'data_for_CapPunkt' 

    config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
    config.model.train_ds.tokens_in_batch = 1024
    config.model.train_ds.text_file = 'text_train.txt'
    config.model.train_ds.labels_file = 'labels_train.txt'
    config.model.train_ds.audio_file = 'audio_train.txt'
    config.model.train_ds.num_workers = 32
    config.model.validation_ds.tokens_in_batch = 1024
    config.model.validation_ds.text_file = 'text_dev.txt'
    config.model.validation_ds.labels_file = 'labels_dev.txt'
    config.model.validation_ds.audio_file = 'audio_dev.txt'
    config.model.validation_ds.num_workers = 32
    config.model.test_ds.tokens_in_batch = 1024
    config.model.test_ds.text_file = 'text_test.txt'
    config.model.test_ds.labels_file = 'labels_test.txt'
    config.model.test_ds.audio_file = 'audio_test.txt'
    config.model.test_ds.num_workers = 32
    config.model.optim.lr = 0.00002
    config.model.audio_encoder.pretrained_model = PRETRAINED_ASR_MODEL
    config.model.train_ds.preload_audios = True
    config.model.validation_ds.preload_audios = True
    config.model.test_ds.preload_audios = True
    
    if restore_model_path == None:
#         config = OmegaConf.create(config)
        model = nemo_nlp.models.PunctuationCapitalizationLexicalAudioModel(cfg=config.model, 
                                                                           trainer=trainer)
    else:
#         config = OmegaConf.create(config)
        model = nemo_nlp.models.PunctuationCapitalizationLexicalAudioModel.restore_from(restore_model_path, 
                                                                                       trainer=trainer)
        model.setup_test_data(config.model.test_ds)
    
    return trainer, model

In [ ]:
trainer, model = build_trainer_and_model()
trainer.fit(model)

In [5]:
model.save_to('NLP_models/LexCapPunkt100epochs.nemo')

### Test saved model

5 epochs

In [17]:
trainer, pretrained_model = build_trainer_and_model('NLP_models/LexCapPunkt5epochs.nemo')
trainer.test(pretrained_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

[NeMo I 2022-12-15 12:28:10 punctuation_capitalization_model:333] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         97.42      98.33      97.87      66419
    ! (label_id: 1)                                         46.53      17.18      25.09        390
    , (label_id: 2)                                         78.98      69.86      74.14       5321
    . (label_id: 3)                                         93.86      96.73      95.27       8385
    ? (label_id: 4)                                         70.45      68.38      69.40        544
    -------------------
    micro avg                                               95.70      95.70      95.70      81059
    macro avg                                               77.45      70.10      72.36      81059
    weighted avg                                            95.42      95.70      95.51  

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_capit_f1       │     97.73580932617188     │
│   test_capit_precision    │     98.07246398925781     │
│     test_capit_recall     │     97.40682220458984     │
│         test_loss         │    0.2501354560660227     │
│       test_punct_f1       │     72.35643005371094     │
│   test_punct_precision    │     77.44862365722656     │
│     test_punct_recall     │     70.09593963623047     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2501354560660227,
  'test_punct_precision': 77.44862365722656,
  'test_punct_f1': 72.35643005371094,
  'test_punct_recall': 70.09593963623047,
  'test_capit_precision': 98.07246398925781,
  'test_capit_f1': 97.73580932617188,
  'test_capit_recall': 97.40682220458984}]

20 epochs

In [19]:
trainer, pretrained_model = build_trainer_and_model('NLP_models/LexCapPunkt20epochs.nemo')
trainer.test(pretrained_model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


[NeMo I 2022-12-15 12:28:59 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpuxbqgrah/35346f4f44504a64b606693320ef3afc_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-12-15 12:29:03 modelPT:222] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-12-15 12:29:03 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    ds_item: data_for_CapPunkt
    text_file: text_train.txt
    labels_file: labels_train.txt
    audio_file: audio_train.txt
    use_audio: true
    use_bucketing: true
    preload_audios: true
    tokens_in_batch: 1024
    max_seq_length: 512
    sample_rate: 16000
    num_workers: 32
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    
[NeMo W 2022-12-15 12:29:03 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_

[NeMo I 2022-12-15 12:29:08 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-15 12:29:08 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-15 12:29:08 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-15 12:29:08 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-15 12:29:08 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-15 12:29:08 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-15 12:29:09 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ё
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-12-15 12:29:09 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation da

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

[NeMo I 2022-12-15 12:30:51 punctuation_capitalization_model:333] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         97.76      98.41      98.08      66419
    ! (label_id: 1)                                         50.68      28.72      36.66        390
    , (label_id: 2)                                         81.08      73.60      77.15       5321
    . (label_id: 3)                                         94.37      97.14      95.73       8385
    ? (label_id: 4)                                         74.86      71.69      73.24        544
    -------------------
    micro avg                                               96.13      96.13      96.13      81059
    macro avg                                               79.75      73.91      76.17      81059
    weighted avg                                            95.94      96.13      96.01  

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_capit_f1       │     97.95774841308594     │
│   test_capit_precision    │     98.30320739746094     │
│     test_capit_recall     │     97.62031555175781     │
│         test_loss         │    0.3855004098316153     │
│       test_punct_f1       │     76.17469787597656     │
│   test_punct_precision    │     79.7488784790039      │
│     test_punct_recall     │     73.90983581542969     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.3855004098316153,
  'test_punct_precision': 79.7488784790039,
  'test_punct_f1': 76.17469787597656,
  'test_punct_recall': 73.90983581542969,
  'test_capit_precision': 98.30320739746094,
  'test_capit_f1': 97.95774841308594,
  'test_capit_recall': 97.62031555175781}]

In [23]:
trainer, pretrained_model = build_trainer_and_model('NLP_models/LexCapPunkt10epochs.nemo')
trainer.test(pretrained_model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


[NeMo I 2022-12-15 16:10:17 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpq1907iqm/47aa42504e1542ab82a88528c590fff3_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-12-15 16:10:21 modelPT:222] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-12-15 16:10:21 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    ds_item: data_for_CapPunkt
    text_file: text_train.txt
    labels_file: labels_train.txt
    audio_file: audio_train.txt
    use_audio: true
    use_bucketing: true
    preload_audios: true
    tokens_in_batch: 1024
    max_seq_length: 512
    sample_rate: 16000
    num_workers: 32
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    
[NeMo W 2022-12-15 16:10:21 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_

[NeMo I 2022-12-15 16:10:25 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-15 16:10:25 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-15 16:10:25 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-15 16:10:26 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-15 16:10:26 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-15 16:10:26 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-15 16:10:26 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ё
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-12-15 16:10:26 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation da

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

[NeMo I 2022-12-15 16:12:55 punctuation_capitalization_model:333] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         97.79      98.24      98.02      66419
    ! (label_id: 1)                                         49.05      26.41      34.33        390
    , (label_id: 2)                                         79.26      74.05      76.56       5321
    . (label_id: 3)                                         94.19      97.08      95.61       8385
    ? (label_id: 4)                                         73.83      69.49      71.59        544
    -------------------
    micro avg                                               96.00      96.00      96.00      81059
    macro avg                                               78.82      73.05      75.22      81059
    weighted avg                                            95.81      96.00      95.88  

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_capit_f1       │     97.79620361328125     │
│   test_capit_precision    │     97.96492004394531     │
│     test_capit_recall     │     97.6294174194336      │
│         test_loss         │    0.3256343603444133     │
│       test_punct_f1       │     75.22386932373047     │
│   test_punct_precision    │     78.82410430908203     │
│     test_punct_recall     │     73.05257415771484     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.3256343603444133,
  'test_punct_precision': 78.82410430908203,
  'test_punct_f1': 75.22386932373047,
  'test_punct_recall': 73.05257415771484,
  'test_capit_precision': 97.96492004394531,
  'test_capit_f1': 97.79620361328125,
  'test_capit_recall': 97.6294174194336}]

In [7]:
trainer, pretrained_model = build_trainer_and_model('NLP_models/LexCapPunkt50epochs.nemo')
trainer.test(pretrained_model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


[NeMo I 2022-12-16 17:16:28 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpa5pi62o3/dce82fb4703f445d997b8014ea5075f3_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-12-16 17:16:31 modelPT:222] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-12-16 17:16:32 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    ds_item: data_for_CapPunkt
    text_file: text_train.txt
    labels_file: labels_train.txt
    audio_file: audio_train.txt
    use_audio: true
    use_bucketing: true
    preload_audios: true
    tokens_in_batch: 1024
    max_seq_length: 512
    sample_rate: 16000
    num_workers: 32
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    
[NeMo W 2022-12-16 17:16:32 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_

[NeMo I 2022-12-16 17:16:36 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-16 17:16:36 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-16 17:16:36 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-16 17:16:36 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-16 17:16:36 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-16 17:16:36 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-16 17:16:37 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ё
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-12-16 17:16:37 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation da

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

[NeMo I 2022-12-16 17:18:51 punctuation_capitalization_model:333] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         97.61      98.46      98.03      66419
    ! (label_id: 1)                                         48.47      28.46      35.86        390
    , (label_id: 2)                                         80.86      72.04      76.20       5321
    . (label_id: 3)                                         94.44      96.99      95.70       8385
    ? (label_id: 4)                                         75.89      66.54      70.91        544
    -------------------
    micro avg                                               96.02      96.02      96.02      81059
    macro avg                                               79.45      72.50      75.34      81059
    weighted avg                                            95.80      96.02      95.88  

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_capit_f1       │     97.87837219238281     │
│   test_capit_precision    │     98.38262176513672     │
│     test_capit_recall     │     97.39103698730469     │
│         test_loss         │    0.3839734835710401     │
│       test_punct_f1       │     75.34028625488281     │
│   test_punct_precision    │     79.45431518554688     │
│     test_punct_recall     │     72.49938201904297     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.3839734835710401,
  'test_punct_precision': 79.45431518554688,
  'test_punct_f1': 75.34028625488281,
  'test_punct_recall': 72.49938201904297,
  'test_capit_precision': 98.38262176513672,
  'test_capit_f1': 97.87837219238281,
  'test_capit_recall': 97.39103698730469}]

In [6]:
trainer, pretrained_model = build_trainer_and_model('NLP_models/LexCapPunkt100epochs.nemo')
trainer.test(pretrained_model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


[NeMo I 2022-12-17 15:55:48 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmp6p76y2vf/45687d6462d34f2482e77c84653582a4_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-12-17 15:55:52 modelPT:222] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-12-17 15:55:52 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    ds_item: data_for_CapPunkt
    text_file: text_train.txt
    labels_file: labels_train.txt
    audio_file: audio_train.txt
    use_audio: true
    use_bucketing: true
    preload_audios: true
    tokens_in_batch: 1024
    max_seq_length: 512
    sample_rate: 16000
    num_workers: 32
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    
[NeMo W 2022-12-17 15:55:52 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_

[NeMo I 2022-12-17 15:55:56 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-17 15:55:56 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-17 15:55:56 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-17 15:55:57 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-17 15:55:57 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-17 15:55:57 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-17 15:55:58 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ё
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-12-17 15:55:58 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation da

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

[NeMo I 2022-12-17 15:58:12 punctuation_capitalization_model:333] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         97.47      98.42      97.94      66419
    ! (label_id: 1)                                         35.67      30.00      32.59        390
    , (label_id: 2)                                         80.28      70.55      75.10       5321
    . (label_id: 3)                                         94.52      95.52      95.02       8385
    ? (label_id: 4)                                         68.68      64.89      66.73        544
    -------------------
    micro avg                                               95.73      95.73      95.73      81059
    macro avg                                               75.32      71.87      73.48      81059
    weighted avg                                            95.54      95.73      95.61  

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_capit_f1       │     97.8282470703125      │
│   test_capit_precision    │     98.11518859863281     │
│     test_capit_recall     │     97.54688262939453     │
│         test_loss         │    0.34236304122560324    │
│       test_punct_f1       │     73.47581481933594     │
│   test_punct_precision    │     75.32382202148438     │
│     test_punct_recall     │      71.874755859375      │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.34236304122560324,
  'test_punct_precision': 75.32382202148438,
  'test_punct_f1': 73.47581481933594,
  'test_punct_recall': 71.874755859375,
  'test_capit_precision': 98.11518859863281,
  'test_capit_f1': 97.8282470703125,
  'test_capit_recall': 97.54688262939453}]